
# Inflation

Inflation model for analyzing, tracking, and predicting components of inflation.

We will have [CPI](https://www.bls.gov/cpi/) [CPI-U](https://beta.bls.gov/dataViewer/view/timeseries/CUUR0000SA0) [CPI-W](https://beta.bls.gov/dataViewer/view/timeseries/CWUR0000SA0) [CES](https://data.bls.gov/cex/)

 has features merged into a single datframe derived from a custom column of series id, month/year, value

In [3]:
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split

In [ ]:
# Read in our datasets
cpi =   pd.read_csv('')
cpi_u = pd.read_csv('')
cpi_w = pd.read_csv('')
ces =   pd.read_csv('')

**Create a custom dataframe**

**Get the summary statistics** 